## Data Analysis with SQLite

This is a SQLite project I am working on to get used to SQLite. The data is from a bikeshare comapny called *Hubway*, which has information on over 1.5 million trips taken by customers in the Boston area, as well data on all of the stations in the Boston area. The data is in a database and I've chosen to use SQLite instead of SQL because I plan to make some plots of the data using matplotlib and seaborn in the next section. SQLite has most of the useful functionalities of SQL and I think it's used when trying to get a quick summarizing analysis of the data within a database. A link to the database is here: In this section I'll answer a number of questions including:

* What type of customers tke the longest trips?
* How long do most trips last?
* Which stations receive trips the most frequently?
* Do customers return to the same station they started at often?



The first thing we'd like to do is import SQLite:

In [7]:
import sqlite3 as lite
import pandas as pd

Now we can connect to the SQL database:

In [8]:
con = lite.connect('hubway.db')

This gives us access to Hubway's database, but we need to figure out the parameters that are in it. We can 
make a fucntion that will output any type of query:

In [9]:
def sql_query(query, database):
    return pd.read_sql_query(query,database)

With this it is possible to run any query in a manner very similar to how it would be done in SQL. We can 
take a look at the first five rows of data from the table 'trips' as an example:

In [61]:
query = 'SELECT * FROM trips LIMIT 5;'
sql_query(query, con)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976.0,Male
1,2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966.0,Male
2,3,56,2011-07-28 10:33:00,23,2011-07-28 10:34:00,23,B00456,Registered,'02108,1943.0,Male
3,4,64,2011-07-28 10:35:00,23,2011-07-28 10:36:00,23,B00554,Registered,'02116,1981.0,Female
4,5,12,2011-07-28 10:37:00,23,2011-07-28 10:37:00,23,B00554,Registered,'97214,1983.0,Female


Looking at these rows of data, we can automatically see that there at least two distinct values for every column 
(e.g 'end_station' is 23 and 40) except for sub_type. We can check the total number of different types of sub_types 
using the query below:

In [62]:
query = '''
        SELECT sub_type AS "Customer Type", 
        COUNT(*) AS "Count" 
        FROM trips 
        GROUP BY sub_type;
        '''

sql_query(query, con)


,Customer Type,Count
0,Casual,464809
1,Registered,1105192


So customers are either described as Registered or not. Most of the trips seem to be had by registered riders, but
let's take a look at how long these rides last:

In [63]:
query = '''
        SELECT sub_type AS "Customer Type", 
        AVG(duration) AS "Average Duration" 
        FROM trips 
        GROUP BY sub_type;
        '''

sql_query(query, con)

,Customer Type,Average Duration
0,Casual,1519.643897
1,Registered,657.026067


We're starting to unveil a story here. The bulk of rides are taken by registered customers but rides by casual 
customers tend to be considerably longer. Let's look at the type of customers who had the ten longest ride times:

In [64]:
query = '''
        SELECT duration, sub_type AS "Customer Type"
        FROM trips 
        ORDER BY duration DESC LIMIT 10;
        '''

sql_query(query, con)

,duration,Customer Type
0,9999,Casual
1,9998,Casual
2,9998,Casual
3,9997,Casual
4,9996,Casual
5,9996,Casual
6,9995,Casual
7,9995,Registered
8,9994,Casual
9,9994,Casual


Out of the ten customers with the longest rides only one was registered! Along with the customer type, let's also 
look at *when* the ten longest rides were taken as well as their gender:

In [65]:
query = '''
        SELECT duration, 
        sub_type AS "Customer Type",
        gender,
        start_date
        FROM trips 
        ORDER BY duration DESC LIMIT 10;
        '''

sql_query(query, con)

,duration,Customer Type,gender,start_date
0,9999,Casual,,2012-09-26 18:34:00
1,9998,Casual,,2011-08-09 10:42:00
2,9998,Casual,,2012-08-15 12:11:00
3,9997,Casual,,2011-10-09 02:30:00
4,9996,Casual,,2011-11-19 13:48:00
5,9996,Casual,,2012-03-23 14:48:00
6,9995,Casual,,2011-08-20 12:20:00
7,9995,Registered,Male,2012-07-03 18:28:00
8,9994,Casual,,2011-08-03 17:16:00
9,9994,Casual,,2011-09-14 15:44:00


Looking at the ten longest rides it seems that most of them were taken in either July, August or September. We can also see that it seems like information on gender is only available for registered customers. The story is begining to seem like many casual riders take long rides during the summer. I'll look into this further in the data visualization section of my analysis, where it is a lot simpler to visualize the *entire* data set. Since we've verified that casual customers take longer rides than members, we can continue on to our next question: "Is there a correlation between the *length* of the ride and *age* of the customer?". This can be answered by calculating the ride length for customers *under* thirty and customers *over* thirty. We can then group this by gender:

In [66]:
query = '''
        SELECT AVG(duration) AS "Trip length for customers under 30",
        gender as "Gender", 
        COUNT(*) as "Count"
        FROM trips 
        WHERE (2017 - birth_date) < 31 
        GROUP BY gender;
        '''

sql_query(query, con)

,Trip length for customers under 30,Gender,Count
0,712.263291,Female,18208
1,614.851432,Male,41577


In [67]:
query = '''
        SELECT AVG(duration) AS "Trip length for customers over 30",
        gender as "Gender", 
        COUNT(*) as "Count"
        FROM trips 
        WHERE (2017 - birth_date) > 30 AND sub_type = 'Registered'
        GROUP BY gender; 
        '''

sql_query(query, con)

,Trip length for customers over 30,Gender,Count
0,740.070135,Female,253125
1,420.000000,Female\n,1
2,631.438482,Male,792281


So old people are taking the longer trips! 923 seconds is about 43% longer than 644 seconds, so folks over 30 tend to take considerably longer trips than those under 30. Women spend ~ 1 minute more riding than men but males make up 76% of the registered customers. Another interesting question we can ask is, "How long are trips where customers are ending their rides at the same station they started at"?:


In [68]:
query = '''
        SELECT COUNT(*), 
        AVG(duration) AS "Trip length"
        FROM trips
        WHERE trips.start_station = trips.end_station
        '''
sql_query(query, con)

,COUNT(*),Trip length
0,73068,2109.498303


2109 seconds is ~ 35 minutes. We can compare this to the average trip duration:

In [69]:
query = '''
        SELECT AVG(duration) AS "Average Trip length"
        FROM trips
        '''
sql_query(query, con)

,Average Trip length
0,912.409682


912 seconds is ~ 15 minutes, so trips where the return station is the same as the start station tend to 
last more than twice as long as other regular trips. Another interesting question taht we can ask is which stations to customers *start* and *end* their trips at the most? To answer this we will have to join data from the 'trips' table with data from the 'stations' table: 


In [70]:
query = '''
SELECT stations.station AS "Station", COUNT(*) AS "Count"
FROM trips 
JOIN stations
ON trips.start_station = stations.id
WHERE trips.start_station = trips.end_station
GROUP BY stations.station
ORDER BY COUNT(*) DESC
LIMIT 10;
'''

sql_query(query, con)

,Station,Count
0,The Esplanade - Beacon St. at Arlington St.,3064
1,Charles Circle - Charles St. at Cambridge St.,2739
2,Boston Public Library - 700 Boylston St.,2548
3,Boylston St. at Arlington St.,2163
4,Beacon St / Mass Ave,2144
5,Newbury St / Hereford St,1636
6,Prudential Center / Belvidere,1590
7,South Station - 700 Atlantic Ave.,1517
8,Christian Science Plaza,1482
9,Kenmore Sq / Comm Ave,1439


Let's have a look at what else is in the stations data by peeking at a few rows:

In [71]:
query = ''' 
        SELECT * 
        FROM stations LIMIT 5; 
        '''

sql_query(query, con)

,id,station,municipality,lat,lng
0,3,Colleges of the Fenway,Boston,42.340021,-71.100812
1,4,Tremont St. at Berkeley St.,Boston,42.345392,-71.069616
2,5,Northeastern U / North Parking Lot,Boston,42.341814,-71.090179
3,6,Cambridge St. at Joy St.,Boston,42.361284999999995,-71.06514
4,7,Fan Pier,Boston,42.353412,-71.044624


We can see that the'stations' table holds the id, station name, municipality and position ('lat' and 'lng') of 
for every station. We can join data from this table to the trips table in order to answer questions such as "Which 
stations get the most trips?"

In [72]:
query = '''
        SELECT stations.station AS "Station", 
        COUNT(*) AS "Count"
        FROM trips 
        JOIN stations
        ON trips.start_station = stations.id
        GROUP BY stations.station
        ORDER BY COUNT(*) DESC
        LIMIT 10;
        '''

sql_query(query, con)

,Station,Count
0,South Station - 700 Atlantic Ave.,56123
1,Boston Public Library - 700 Boylston St.,41994
2,Charles Circle - Charles St. at Cambridge St.,35984
3,Beacon St / Mass Ave,35275
4,MIT at Mass Ave / Amherst St,33644
5,Back Bay / South End Station,32677
6,Boylston St. at Arlington St.,32410
7,Kenmore Sq / Comm Ave,30835
8,The Esplanade - Beacon St. at Arlington St.,28119
9,Newbury St / Hereford St,26733


We can also look at the the 10 *least* frequented stations:

In [73]:
query = '''
        SELECT stations.station AS "Station", 
        COUNT(*) AS "Count"
        FROM trips 
        JOIN stations
        ON trips.start_station = stations.id
        GROUP BY stations.station
        ORDER BY COUNT(*) ASC
        LIMIT 10;
        '''

sql_query(query, con)

,Station,Count
0,Upham's Corner - Ramsey St at Dudley St,49
1,E. Cottage St at Columbia Rd,91
2,Egleston Square at Columbus Ave,161
3,Jackson Square T at Centre St,200
4,Summer St at Cutter St,219
5,Hayes Square at Vine St.,234
6,Harvard University River Houses at DeWolfe St...,268
7,JP Monument - South St at Centre St,305
8,Green St T,309
9,JP Centre - Centre Street at Myrtle Street,325


#### Summary

So far we've learned a lot about the customers of Hubway, mainly that:
* Most customers are registered members
* 76% of registered customers are male
* Registered female customers rent bikes for 1 minute longer than males do
* Casual customers take trips that last more than twice as long as registered customers
* Customers over the age of thirty take trips which are ~ 40% longer than customers under thirty
* Certain staions (South Station, Boston Public Library) are strongly preferred over others (Upham's Corner)
* There *may* be a correlation between sub_type and the time of year a customer rents a bike. 


Moving forward there are a lot more questions we can ask using this data including:
* What time of the year do bike rentals rise?
* How much more are stations in densley packed areas frequented than those in sparsley packed areas, and during what 
time of the day, week, year is the disparity the largest?
* Can we predict if a customer will become a member based on *where* and *when* they rent bikes?

These are questions which, in my opinion can be better answered and visualized using pandas, matplotlib, seaborn and scikit-learn. I will answer these questions and more in the next section!
